In [1]:
import numpy as np
from Network import Network
import struct
from array import array

In [2]:
def MSE(y, Y):
    return np.mean((y-Y)**2)

In [3]:
def load(path_img, path_lbl):
    with open(path_lbl, 'rb') as file:
        magic, size = struct.unpack(">II", file.read(8))
        if magic != 2049:
            raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
        labels = array("B", file.read())

    with open(path_img, 'rb') as file:
        magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
        
        if magic != 2051:
            raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
        
        image_data = array("B", file.read())

    images = np.full((size, rows * cols), 0.0)

    for i in range(size):
        images[i][:] = image_data[i * rows * cols:(i + 1) * rows * cols]
    return images, labels

In [4]:
def to_categorical(y, num_classes=None):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [5]:
train_img_path = "./lecun-mnist/train-images.idx3-ubyte"
train_labels_path = "./lecun-mnist/train-labels.idx1-ubyte"
imgs, labels = load(train_img_path, train_labels_path)

In [6]:
X = imgs
target = to_categorical(labels, 10)

In [7]:
net = Network(X.shape[1])

In [8]:
net.add_layer(300)

In [9]:
net.add_layer(target.shape[1])

In [10]:
net.fit(X, target, 20)

Layer.py:28: RuntimeWarning: overflow encountered in exp
  prediction = 1 / (1 + np.exp(-z))


In [11]:
MSE(net.predict(X), labels)

10.77875

In [13]:
train_img_path = "./lecun-mnist/t10k-images.idx3-ubyte"
train_labels_path = "./lecun-mnist/t10k-labels.idx1-ubyte"
test_imgs, test_labels = load(train_img_path, train_labels_path)
X_test = test_imgs
y_test = test_labels

In [14]:
MSE(net.predict(X_test), y_test)

10.884600000000001